<div>
<table border="1" style="border-collapse: collapse; width: 100%;">
<tbody>
<tr>
<td style="width: 100%; text-align: center;" colspan="4">
<h1><em><span style="color: #808080;"><strong>PROJET 9 - </strong></span></em><em><span style="color: #808080;"><strong>Parcours Ing&eacute;nieur IA</strong></span></em></h1>
</td>
</tr>
<tr style="text-align: center;">
<td colspan="4" style="width: 100%;"><img src="https://user.oc-static.com/upload/2019/10/24/15719078448088_Capture%20d%E2%80%99e%CC%81cran%202019-10-24%20a%CC%80%2010.50.32.png" alt="Logo My Content"></td>
</tr>
<tr>
<td style="width: 100%; text-align: center;" colspan="4">
<h1><span style="color: #808080;"><strong><em>Réalisez une application de recommandation de contenu</em></strong></span></h1>
</td>
</tr>
<tr>
<td style="width: 50%; text-align: center;" colspan="2"><img src="https://consent.trustarc.com/get?name=oc_logo.png" width="250px" /></td>
<td style="width: 50%; text-align: center;" colspan="2"><img src="Microsoft-Azure-Logo.png" width="250px" /></td>
</tr>
</tbody>
</table>
</div>

<hr /><h1 style="text-align: center;"><span style="color: #666699;">📤<em><strong> Modélisation et Upload Azure CosmosDB </strong></em></span></h1><hr />

In [1]:
import os
import sys

# Add source directory to python path
sys.path.append(os.path.abspath("../"))

from datetime import datetime
from pathlib import Path
from azure.cosmos import CosmosClient, PartitionKey

import pandas as pd
import implicit
from scipy import sparse
from tqdm import tqdm
from cfg import AZURE_COSMOSDB_URI, AZURE_COSMOSDB_KEY


# Plotly as Pandas plotting backend
pd.options.plotting.backend = "plotly"

In [2]:
DATA_PATH = "../../data/news-portal-user-interactions-by-globocom"

<h2 style="text-align: left;"><span style="color: #666699;"><em><strong>Chargement de **Clicks Data**</strong></em></span></h2>

In [3]:
if 'clicks.csv' in os.listdir(os.getcwd()):
    clicks = pd.read_csv('clicks.csv')
    print('clicks dataFrame ready...\n')
    clicks.info()

else :
    clicks = pd.concat(
        [
            pd.read_csv(
                click_file_path,
                parse_dates=["session_start", "click_timestamp"],
                date_parser=lambda x: datetime.fromtimestamp(int(int(x) / 1000)),
                dtype={
                    "user_id": "category",
                    "session_id": "category",
                    "session_size": "int",
                    "click_article_id": "category",
                    "click_environment": "category",
                    "click_deviceGroup": "category",
                    "click_os": "category",
                    "click_country": "category",
                    "click_region": "category",
                    "click_referrer_type": "category",
                },
            ).replace(
                {
                    "click_environment": {
                        "1": "1 - Facebook Instant Article",
                        "2": "2 - Mobile App",
                        "3": "3 - AMP (Accelerated Mobile Pages)",
                        "4": "4 - Web",
                    },
                    "click_deviceGroup": {
                        "1": "1 - Tablet",
                        "2": "2 - TV",
                        "3": "3 - Empty",
                        "4": "4 - Mobile",
                        "5": "5 - Desktop",
                    },
                    "click_os": {
                        "1": "1 - Other",
                        "2": "2 - iOS",
                        "3": "3 - Android",
                        "4": "4 - Windows Phone",
                        "5": "5 - Windows Mobile",
                        "6": "6 - Windows",
                        "7": "7 - Mac OS X",
                        "8": "8 - Mac OS",
                        "9": "9 - Samsung",
                        "10": "10 - FireHbbTV",
                        "11": "11 - ATV OS X",
                        "12": "12 - tvOS",
                        "13": "13 - Chrome OS",
                        "14": "14 - Debian",
                        "15": "15 - Symbian OS",
                        "16": "16 - BlackBerry OS",
                        "17": "17 - Firefox OS",
                        "18": "18 - Android",
                        "19": "19 - Brew MP",
                        "20": "20 - Chromecast",
                        "21": "21 - webOS",
                        "22": "22 - Gentoo",
                        "23": "23 - Solaris",
                    },
                }
            )
            for click_file_path in tqdm(
                sorted(Path(DATA_PATH, "clicks").glob("clicks_hour_*.csv"))
            )
        ],
        sort=False,
        ignore_index=True,
        verify_integrity=True,
    )

clicks.to_csv('clicks.csv', index=False)

clicks dataFrame ready...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2988181 entries, 0 to 2988180
Data columns (total 12 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   user_id              int64 
 1   session_id           int64 
 2   session_start        object
 3   session_size         int64 
 4   click_article_id     int64 
 5   click_timestamp      object
 6   click_environment    object
 7   click_deviceGroup    object
 8   click_os             object
 9   click_country        int64 
 10  click_region         int64 
 11  click_referrer_type  int64 
dtypes: int64(7), object(5)
memory usage: 273.6+ MB


In [4]:
clicks = clicks.astype(
        {"session_start": "datetime64[ns]", "click_timestamp": "datetime64[ns]"}
    )

<h2 style="text-align: left;"><span style="color: #666699;"><em><strong>Création **Implicit Ratings**</strong></em></span></h2>

In [5]:
ratings = (
    clicks.reset_index()
    .groupby(["user_id", "click_article_id"])
    .agg(
        rating=("index", "count"),
    )
    .reset_index()
)
ratings.head(1)

,user_id,click_article_id,rating
0,0,68866,1


<h2 style="text-align: left;"><span style="color: #666699;"><em><strong>Création **Implicit Ratings CSR Matrix**</strong></em></span></h2>

In [6]:
ratings_sparse = sparse.csr_matrix(
    (
        ratings["rating"],
        (ratings["user_id"].astype("int"), ratings["click_article_id"].astype("int")),
    ),
)
ratings_sparse.shape

(322897, 364047)

<h2 style="text-align: center;"><span style="color: #666699;"><em><strong>Entrainement sur GPU du meilleur modèle **ALS**</strong></em></span></h2>

In [16]:
model = implicit.gpu.als.AlternatingLeastSquares()
model.fit(ratings_sparse)

  0%|          | 0/15 [00:00<?, ?it/s]

<h2 style="text-align: center;"><span style="color: #666699;"><em><strong>Creation CosmosDB client et database</strong></em></span></h2>

In [17]:
client = CosmosClient(AZURE_COSMOSDB_URI, AZURE_COSMOSDB_KEY)
database = client.create_database_if_not_exists(id="Recommendation")
container = database.create_container_if_not_exists(
    id="UserArticles",
    partition_key=PartitionKey(path="/id"),
)

<h2 style="text-align: center;"><span style="color: #666699;"><em><strong>Upload des recommendations vers CosmosDB</strong></em></span></h2>

In [18]:
## Insert recommendations of the only first 100 users into CosmosDB

user_ids = ratings["user_id"].unique().astype("int")
user_ids = user_ids[:100]

recos, _ = model.recommend(user_ids, ratings_sparse[user_ids], N=5)

for i, user_id in tqdm(enumerate(user_ids)):
    container.upsert_item(
        {
            "id": str(user_id),
            "articles": [str(article_id) for article_id in recos[i]],
        }
    )

100it [00:17,  5.83it/s]


<h2 style="text-align: center;"><span style="color: #666699;"><em><strong>Verification CosmosDB</strong></em></span></h2>

In [19]:
def read_items(container):
    print('\n1.3 - Reading all items in a container\n')

    # NOTE: Use MaxItemCount on Options to control how many items come back per trip to the server
    #       Important to handle throttles whenever you are doing operations such as this that might
    #       result in a 429 (throttled request)
    item_list = list(container.read_all_items(max_item_count=10))

    print('Found {0} items'.format(item_list.__len__()))

read_items(container)


1.3 - Reading all items in a container

Found 100 items
